## Importing essential libraries and reading the dataframe.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats

df = pd.read_csv(r'E:\Learning Data Science With Python\Data Science 102 - Handling Missing Values\Fifa21_Data_Cleaned.csv')

## Looking at the top 5 Rows of the Dataset and the datatypes for more info.

In [2]:
df.head()

,Unnamed: 0,ID,Name,LongName,Nationality,Age,↓OVA,POT,Club,Contract,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,0,158023,L. Messi,Lionel Messi,Argentina,33,93,93,FCBarcelona,2004 ~ 2021,...,Medium,Low,5,85,92,91,95,38,65,771.0
1,1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Portugal,35,92,92,Juventus,2018 ~ 2022,...,High,Low,5,89,93,81,89,35,77,562.0
2,2,200389,J. Oblak,Jan Oblak,Slovenia,27,91,93,AtléticoMadrid,2014 ~ 2023,...,Medium,Medium,3,87,92,78,90,52,90,150.0
3,3,192985,K. De Bruyne,Kevin De Bruyne,Belgium,29,91,91,ManchesterCity,2015 ~ 2023,...,High,High,4,76,86,93,88,64,78,207.0
4,4,190871,Neymar Jr,Neymar da Silva Santos Jr.,Brazil,28,91,91,ParisSaint-Germain,2017 ~ 2022,...,High,Medium,5,91,85,86,94,36,59,595.0


In [3]:
df.info()
pd.set_option('display.max_rows',None)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 76 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        18979 non-null  int64  
 1   ID                18979 non-null  int64  
 2   Name              18979 non-null  object 
 3   LongName          18979 non-null  object 
 4   Nationality       18979 non-null  object 
 5   Age               18979 non-null  int64  
 6   ↓OVA              18979 non-null  int64  
 7   POT               18979 non-null  int64  
 8   Club              18979 non-null  object 
 9   Contract          18979 non-null  object 
 10  Positions         18979 non-null  object 
 11  Height            18939 non-null  float64
 12  Weight            18979 non-null  float64
 13  Preferred Foot    18979 non-null  object 
 14  BOV               18979 non-null  int64  
 15  Best Position     18979 non-null  object 
 16  Joined            18979 non-null  object

## Dropping the first index row, as it is unnecessary.

In [4]:
df.drop(columns = 'Unnamed: 0', inplace=True)

In [5]:
df.head()

,ID,Name,LongName,Nationality,Age,↓OVA,POT,Club,Contract,Positions,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,Argentina,33,93,93,FCBarcelona,2004 ~ 2021,"RW, ST, CF",...,Medium,Low,5,85,92,91,95,38,65,771.0
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Portugal,35,92,92,Juventus,2018 ~ 2022,"ST, LW",...,High,Low,5,89,93,81,89,35,77,562.0
2,200389,J. Oblak,Jan Oblak,Slovenia,27,91,93,AtléticoMadrid,2014 ~ 2023,GK,...,Medium,Medium,3,87,92,78,90,52,90,150.0
3,192985,K. De Bruyne,Kevin De Bruyne,Belgium,29,91,91,ManchesterCity,2015 ~ 2023,"CAM, CM",...,High,High,4,76,86,93,88,64,78,207.0
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,Brazil,28,91,91,ParisSaint-Germain,2017 ~ 2022,"LW, CAM",...,High,Medium,5,91,85,86,94,36,59,595.0


## Checking how many null values exist.

In [6]:
df.isna().sum()

ID                      0
Name                    0
LongName                0
Nationality             0
Age                     0
↓OVA                    0
POT                     0
Club                    0
Contract                0
Positions               0
Height                 40
Weight                  0
Preferred Foot          0
BOV                     0
Best Position           0
Joined                  0
Loan Date End       17966
Value                 248
Wage                 3950
Release Clause       1261
Attacking               0
Crossing                0
Finishing               0
Heading Accuracy        0
Short Passing           0
Volleys                 0
Skill                   0
Dribbling               0
Curve                   0
FK Accuracy             0
Long Passing            0
Ball Control            0
Movement                0
Acceleration            0
Sprint Speed            0
Agility                 0
Reactions               0
Balance                 0
Power       

## Beginning the Data Filling Process

### 1) Filling Height Column with Average Height of the football players.

* Footballers are often very similar in height. There are very few anomalies of players who are extremely short or extremely tall that play football. Thus, we can use the average to fill the missing values.

In [7]:
df['Height'] = df['Height'].fillna(np.mean(df['Height']))

In [8]:
df['Height'].isna().sum()

0

### 2) Filling the Loan Date End with Mode of the Loan Date End.

* The Loan Date End column tells us when the loan for that particular player ends. Since we do not have enough information about each specific players' loan, we can assume that the clubs use a standard date. For ex.- Ending of a season. This is where we will be using the 'Mode' calculation to fill the NA values.

In [9]:
df['Loan Date End'] = pd.to_datetime(df['Loan Date End'])

In [10]:
lde_mode = df['Loan Date End'].mode()[0]
df['Loan Date End'] = df['Loan Date End'].fillna(lde_mode)

In [11]:
df['Loan Date End'].isna().sum()

0

### 3) Filling the Missing Values in Value, Wage, and Release Clause by checking their correlation with other variables.

In [12]:
corr = df.corr(numeric_only = True)
print(corr)

                        ID       Age      ↓OVA       POT    Height    Weight  \
ID                1.000000 -0.753413 -0.486968  0.023736 -0.108096 -0.209851   
Age              -0.753413  1.000000  0.466140 -0.269473  0.089879  0.241991   
↓OVA             -0.486968  0.466140  1.000000  0.632166  0.033094  0.148000   
POT               0.023736 -0.269473  0.632166  1.000000 -0.010021 -0.024602   
Height           -0.108096  0.089879  0.033094 -0.010021  1.000000  0.771242   
Weight           -0.209851  0.241991  0.148000 -0.024602  0.771242  1.000000   
BOV              -0.443686  0.401796  0.987149  0.669677  0.022197  0.128597   
Value            -0.132994  0.046592  0.559090  0.531463  0.004101  0.034762   
Wage             -0.223449  0.138506  0.623768  0.510505  0.015383  0.058350   
Release Clause   -0.162467  0.073031  0.624088  0.581977  0.004843  0.040093   
Attacking        -0.180955  0.146765  0.446337  0.284542 -0.363998 -0.275414   
Crossing         -0.156939  0.124639  0.

After checking the correlation matrix, we can see that Value and Release Clause have a high correlation. Checking the excel told me that the Release clause of a player is roughly about 2.5 times their Value. While this is not true in all cases, it is true in almost 80% of the observations I made. Thus, we should fill it accordingly.

Similarly, the Release Clause of a player is roughly about 250-300 times their Value. Thus, we will fill the Wage column with by dividing it with 275 for ease. 

But the missing values in Value exist when there is a missing value in Release Clause as well. Thus, we will fill Release Clause with its average value and we will back track the other columns.

In [13]:
df['Release Clause'] = df['Release Clause'].fillna(np.mean(df['Release Clause']))

In [14]:
df['Release Clause'].isna().sum()

0

In [15]:
df['Value'] = df['Value'].fillna(df['Release Clause']/2.5)

In [16]:
df['Value'].isna().sum()

0

In [17]:
df['Wage'] = df['Wage'].fillna(df['Release Clause']/275)

In [18]:
df['Wage'].isna().sum()

0

Let us run the correlation matrix again to see if we have gone terribly wrong.

In [19]:
corr_2 = df.corr(numeric_only = True)
print(corr_2)

                        ID       Age      ↓OVA       POT    Height    Weight  \
ID                1.000000 -0.753413 -0.486968  0.023736 -0.108096 -0.209851   
Age              -0.753413  1.000000  0.466140 -0.269473  0.089879  0.241991   
↓OVA             -0.486968  0.466140  1.000000  0.632166  0.033094  0.148000   
POT               0.023736 -0.269473  0.632166  1.000000 -0.010021 -0.024602   
Height           -0.108096  0.089879  0.033094 -0.010021  1.000000  0.771242   
Weight           -0.209851  0.241991  0.148000 -0.024602  0.771242  1.000000   
BOV              -0.443686  0.401796  0.987149  0.669677  0.022197  0.128597   
Value            -0.131312  0.043902  0.554560  0.528102  0.004150  0.034340   
Wage             -0.240836  0.159009  0.602724  0.500406  0.017049  0.061637   
Release Clause   -0.158357  0.070622  0.607924  0.562669  0.004682  0.038788   
Attacking        -0.180955  0.146765  0.446337  0.284542 -0.363998 -0.275414   
Crossing         -0.156939  0.124639  0.

Ideally, we have not made much mistakes, the difference is due to the filling of values and is closer to 0.5% or 1% change.

### 4) Filling the missing values in Hits Column with 0

* Hits might be an important value and we cannot use average/median/mode to fill the missing values as it might impact the models that we train inversely. Thus, we will simply replace the missing values with 0.

In [20]:
df['Hits'] = df['Hits'].fillna(0)

In [21]:
df['Hits'].isna().sum()

0

## Checking the Data after filling mising values

In [22]:
df.head()

,ID,Name,LongName,Nationality,Age,↓OVA,POT,Club,Contract,Positions,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,Argentina,33,93,93,FCBarcelona,2004 ~ 2021,"RW, ST, CF",...,Medium,Low,5,85,92,91,95,38,65,771.0
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Portugal,35,92,92,Juventus,2018 ~ 2022,"ST, LW",...,High,Low,5,89,93,81,89,35,77,562.0
2,200389,J. Oblak,Jan Oblak,Slovenia,27,91,93,AtléticoMadrid,2014 ~ 2023,GK,...,Medium,Medium,3,87,92,78,90,52,90,150.0
3,192985,K. De Bruyne,Kevin De Bruyne,Belgium,29,91,91,ManchesterCity,2015 ~ 2023,"CAM, CM",...,High,High,4,76,86,93,88,64,78,207.0
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,Brazil,28,91,91,ParisSaint-Germain,2017 ~ 2022,"LW, CAM",...,High,Medium,5,91,85,86,94,36,59,595.0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 75 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                18979 non-null  int64         
 1   Name              18979 non-null  object        
 2   LongName          18979 non-null  object        
 3   Nationality       18979 non-null  object        
 4   Age               18979 non-null  int64         
 5   ↓OVA              18979 non-null  int64         
 6   POT               18979 non-null  int64         
 7   Club              18979 non-null  object        
 8   Contract          18979 non-null  object        
 9   Positions         18979 non-null  object        
 10  Height            18979 non-null  float64       
 11  Weight            18979 non-null  float64       
 12  Preferred Foot    18979 non-null  object        
 13  BOV               18979 non-null  int64         
 14  Best Position     1897

In [24]:
df.isna().sum()

ID                  0
Name                0
LongName            0
Nationality         0
Age                 0
↓OVA                0
POT                 0
Club                0
Contract            0
Positions           0
Height              0
Weight              0
Preferred Foot      0
BOV                 0
Best Position       0
Joined              0
Loan Date End       0
Value               0
Wage                0
Release Clause      0
Attacking           0
Crossing            0
Finishing           0
Heading Accuracy    0
Short Passing       0
Volleys             0
Skill               0
Dribbling           0
Curve               0
FK Accuracy         0
Long Passing        0
Ball Control        0
Movement            0
Acceleration        0
Sprint Speed        0
Agility             0
Reactions           0
Balance             0
Power               0
Shot Power          0
Jumping             0
Stamina             0
Strength            0
Long Shots          0
Mentality           0
Aggression

## Writing the CSV file for future use.

In [25]:
df.to_csv(r'E:\Learning Data Science With Python\Data Science 102 - Handling Missing Values\Fifa21_Data_Cleaned_Filled.csv')